In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [2]:
import nltk


In [ ]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.  

In [11]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion_base = re.sub(r'Chapter \d+', '', persuasion)
alice_base = re.sub(r'CHAPTER .*', '', alice)

In [12]:
alice = text_cleaner(alice_base)
persuasion = text_cleaner(persuasion_base)
# question: the text cleaner appears to clean punctuation but the below text still includes it...

In [13]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [14]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

0        1
0  (Alice, was, beginning, to, get, very, tired, ...  Carroll
1  (So, she, was, considering, in, her, own, mind...  Carroll
2  (There, was, nothing, so, VERY, remarkable, in...  Carroll
3                                      (Oh, dear, !)  Carroll
4                         (I, shall, be, late, !, ')  Carroll

In [16]:
sentences.iloc[0][0]

Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversation?'

Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [178]:
from collections import Counter
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# POS dict
pos_d = {'VERB':'verb_ct', 'NOUN':'noun_ct', 'ADV':'adv_ct', 'ADP':'adp_ct', 
         'PROPN':'propn_ct', 'ADJ':'adj_ct', 'DET':'det_ct', 'PUNCT':'punct_ct'}

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words, **kwargs):
    
    # sentence stats
    sent_stats = ['comma_ct', 'word_ct', 'adv_ct', 'adp_ct', 'propn_ct', 'adj_ct', 'punct_ct'] #'verb_ct', 'noun_ct','det_ct',
    if 'sent_stats' in kwargs:
        df = pd.DataFrame(columns=list(common_words) + sent_stats)
        df.loc[:, sent_stats] = 0
    else:
        df = pd.DataFrame(columns=common_words)
        
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    print('made it to the loop')
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # add sentence features
        if 'sent_stats' in kwargs:
            commas = 0
            for token in sentence:
                if token.orth_ == ',':
                    commas += 1
            df.loc[i, 'comma_ct'] = commas
                    
            c = Counter([token.pos_ for token in sentence])
            for key in c.keys():
                if key in pos_d.keys():
                    df.loc[i, pos_d[key]] = c[key]
            
            df.loc[i, 'word_ct'] = len([token for token in sentence if (not token.is_punct)])

        # This counter is just to make sure the kernel didn't hang.
        if i % 100 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [179]:
book_smpl_ct = 500

# Create our data frame with features. This can take a while to run.
word_counts2 = bow_features(sentences[:book_smpl_ct], common_words, kwargs= {'sent_stats': True})

subsentences = sentences[-(book_smpl_ct +1):-1].reset_index(drop=True)
word_counts3 = bow_features(subsentences, common_words, kwargs= {'sent_stats': True})

word_counts_ss = pd.DataFrame(np.concatenate([word_counts2, word_counts3], axis = 0), columns = word_counts2.columns)

made it to the loop
Processing row 0
Processing row 100
Processing row 200
Processing row 300
Processing row 400
made it to the loop
Processing row 0
Processing row 100
Processing row 200
Processing row 300
Processing row 400


In [189]:
word_counts_ss = bow_features(sentences, common_words, kwargs= {'sent_stats': True})

made it to the loop
Processing row 0
Processing row 100
Processing row 200
Processing row 300
Processing row 400
Processing row 500
Processing row 600
Processing row 700
Processing row 800
Processing row 900
Processing row 1000
Processing row 1100
Processing row 1200
Processing row 1300
Processing row 1400
Processing row 1500
Processing row 1600
Processing row 1700
Processing row 1800
Processing row 1900
Processing row 2000
Processing row 2100
Processing row 2200
Processing row 2300
Processing row 2400
Processing row 2500
Processing row 2600
Processing row 2700
Processing row 2800
Processing row 2900
Processing row 3000
Processing row 3100
Processing row 3200
Processing row 3300
Processing row 3400
Processing row 3500
Processing row 3600
Processing row 3700
Processing row 3800
Processing row 3900
Processing row 4000
Processing row 4100
Processing row 4200
Processing row 4300
Processing row 4400
Processing row 4500
Processing row 4600
Processing row 4700
Processing row 4800
Processing r

In [185]:
book_smpl_ct = 500

# Create our data frame with features. This can take a while to run.
word_counts2 = bow_features(sentences[:book_smpl_ct], common_words)

subsentences = sentences[-(book_smpl_ct +1):-1].reset_index(drop=True)
word_counts3 = bow_features(subsentences, common_words)

word_counts = pd.DataFrame(np.concatenate([word_counts2, word_counts3], axis = 0), columns = word_counts2.columns)

made it to the loop
Processing row 0
Processing row 100
Processing row 200
Processing row 300
Processing row 400
made it to the loop
Processing row 0
Processing row 100
Processing row 200
Processing row 300
Processing row 400


In [190]:
word_counts = bow_features(sentences, common_words)


made it to the loop
Processing row 0
Processing row 100
Processing row 200
Processing row 300
Processing row 400
Processing row 500
Processing row 600
Processing row 700
Processing row 800
Processing row 900
Processing row 1000
Processing row 1100
Processing row 1200
Processing row 1300
Processing row 1400
Processing row 1500
Processing row 1600
Processing row 1700
Processing row 1800
Processing row 1900
Processing row 2000
Processing row 2100
Processing row 2200
Processing row 2300
Processing row 2400
Processing row 2500
Processing row 2600
Processing row 2700
Processing row 2800
Processing row 2900
Processing row 3000
Processing row 3100
Processing row 3200
Processing row 3300
Processing row 3400
Processing row 3500
Processing row 3600
Processing row 3700
Processing row 3800
Processing row 3900
Processing row 4000
Processing row 4100
Processing row 4200
Processing row 4300
Processing row 4400
Processing row 4500
Processing row 4600
Processing row 4700
Processing row 4800
Processing r

## Splitting out train/test splits

In [181]:
from sklearn.model_selection import train_test_split

def make_train_test(word_count_df):
    Y = word_count_df['text_source']
    X = word_count_df.iloc[:, ~word_count_df.columns.isin(['text_sentence','text_source'])]

    X_train, X_test, y_train, y_test = train_test_split(X, 
                                                        Y,
                                                        test_size=0.3,
                                                        random_state=0)
    return X_train, X_test, y_train, y_test

In [191]:
X_train_ss, X_test_ss, y_train_ss, y_test_ss = make_train_test(word_counts_ss)

In [186]:
X_train, X_test, y_train, y_test = make_train_test(word_counts)

## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [183]:
from sklearn.linear_model import LogisticRegression

def log_reg(X_train, X_test, y_train, y_test):
    lr = LogisticRegression()
    train = lr.fit(X_train, y_train)
    print(X_train.shape, y_train.shape)
    print('Training set score:', lr.score(X_train, y_train))
    print('\nTest set score:', lr.score(X_test, y_test))
    y_pred = train.predict(X_test)
    print(pd.crosstab(y_test, y_pred))
    return train

In [187]:
lr_ss = log_reg(X_train_ss, X_test_ss, y_train_ss, y_test_ss)

(700, 3062) (700,)
Training set score: 0.9714285714285714

Test set score: 0.9066666666666666
col_0        Austen  Carroll
text_source                 
Austen          142       14
Carroll          14      130


/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [188]:
lr = log_reg(X_train, X_test, y_train, y_test)

(700, 3062) (700,)
Training set score: 0.9714285714285714

Test set score: 0.9066666666666666
col_0        Austen  Carroll
text_source                 
Austen          142       14
Carroll          14      130


/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data:

In [10]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [11]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [12]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [13]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
done


In [14]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.703466666667


col_0    Austen  Carroll
row_0                   
Austen     1158       21
Carroll     535      161

Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%. 

### Work-up:  

With this in mind, I added features for total word count (including stop and common words), comma count, and part of speech count for various parts of speech.

In [153]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(700, 3062) (700,)
Training set score: 0.9714285714285714

Test set score: 0.9066666666666666


/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [154]:
# another work:
paradise_base = gutenberg.raw('milton-paradise.txt')
paradise_base = re.sub(r'VOLUME \w+', '', paradise_base)
paradise_base = re.sub(r'CHAPTER \w+', '', paradise_base)
paradise = text_cleaner(paradise_base)
print(paradise[:100])

Book I Of Man's first disobedience, and the fruit Of that forbidden tree whose mortal taste Brought 


In [155]:
# Parse our cleaned data.
paradise_doc = nlp(paradise)

In [156]:
# Group into sentences.
paradise_sents = [[sent, "Not"] for sent in paradise_doc.sents]

In [157]:
# Nnormalize to the same length or Alice
paradise_sents = paradise_sents[0:len(alice_sents)]

In [158]:
# Build a new Bag of Words data frame for Paradise Lost word counts.
# We'll use the same common words from Alice and Persuasion.
paradise_sentences = pd.DataFrame(paradise_sents)
paradise_bow = bow_features(paradise_sentences[:book_smpl_ct], common_words)

print('done')

made it to the loop
Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450
done


### Alice v. any other work  

In this case, the `Austen` label will be interpretted as **not** `Carroll`, so the test data will be prepped with `Austen` labels, added to the `Carroll` test set and scored.


In [159]:
# filter out the Carroll test rows
X_test['target'] = y_test
test_Carroll = X_test[X_test.target == 'Carroll']

X_test_Carroll = test_Carroll.iloc[:, ~test_Carroll.columns.isin(['target'])] #X_test[y_test[y_test == 'Carroll'].index]
y_test_Carroll = test_Carroll['target']

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [160]:
# prep the non Carroll (Austen) test rows
nonCarroll_bow = paradise_bow
nonCarroll_bow['text_source'] = ['Austen' for ik in range(len(nonCarroll_bow))]

subsample_nonCarroll = nonCarroll_bow.sample(n=len(y_test_Carroll))
X_test_nonCarroll = subsample_nonCarroll.iloc[:, ~subsample_nonCarroll.columns.isin(['text_sentence','text_source']) ]
y_test_nonCarroll = subsample_nonCarroll['text_source']

In [161]:
# concatenate test sets to full Carroll-nonCarroll test set
X_test_CnonC = np.concatenate([X_test_Carroll, X_test_nonCarroll], axis=0)
y_test_CnonC = np.concatenate([y_test_Carroll, y_test_nonCarroll], axis=0)

In [162]:
# run existing model over new test set
print('\nTest set score:', lr.score(X_test_CnonC, y_test_CnonC))
lr_CnC_predicted = lr.predict(X_test_CnonC)
pd.crosstab(y_test_CnonC, lr_CnC_predicted)


Test set score: 0.7604166666666666


col_0    Austen  Carroll
row_0                   
Austen       89       55
Carroll      14      130

### Persuasion v. any other work

In [163]:
# filter out the Persuasion test rows
X_test['target'] = y_test
test_Pers = X_test[X_test.target == 'Austen']

X_test_Pers = test_Pers.iloc[:, ~test_Pers.columns.isin(['target'])] #X_test[y_test[y_test == 'Carroll'].index]
y_test_Pers = test_Pers['target']


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [164]:
# prep the non Austen (Carroll) test rows
nonPers_bow = paradise_bow
nonPers_bow['text_source'] = ['Carroll' for ik in range(len(nonPers_bow))]

subsample_nonPers = nonPers_bow.sample(n=len(y_test_Pers))
X_test_nonPers = subsample_nonPers.iloc[:, ~subsample_nonPers.columns.isin(['text_sentence','text_source']) ]
y_test_nonPers = subsample_nonPers['text_source']

In [165]:
# concatenate test sets to full Carroll-nonCarroll test set
X_test_PnonP = np.concatenate([X_test_Pers, X_test_nonPers], axis=0)
y_test_PnonP = np.concatenate([y_test_Pers, y_test_nonPers], axis=0)

In [166]:
# run existing model over new test set
print('\nTest set score:', lr.score(X_test_PnonP, y_test_PnonP))
lr_PnP_predicted = lr.predict(X_test_PnonP)
pd.crosstab(y_test_PnonP, lr_PnP_predicted)


Test set score: 0.6634615384615384


col_0    Austen  Carroll
row_0                   
Austen      142       14
Carroll      91       65

### Austen v. any other work

In [167]:
# another work:
emma_base = gutenberg.raw('austen-emma.txt')
emma_base = re.sub(r'VOLUME \w+', '', emma_base)
emma_base = re.sub(r'CHAPTER \w+', '', emma_base)
emma = text_cleaner(emma_base)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [168]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [169]:
# Group into sentences.
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]


In [170]:
# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]


In [171]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences[:book_smpl_ct], common_words)

print('done')

made it to the loop
Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450
done


In [172]:
# build smaller subsamples of Austen works so that the test set will be half Austen, half not Austen (in this case, Milton)
subsample_emma = emma_bow.sample(n=int(np.floor(len(y_test_Carroll)/2)))
X_test_emma = subsample_emma.iloc[:, ~subsample_emma.columns.isin(['text_sentence','text_source']) ]
y_test_emma = subsample_emma['text_source']

subsample_Pers = test_Pers.sample(n=len(y_test_Pers)-len(y_test_emma))
X_test_Pers = subsample_Pers.iloc[:, ~subsample_Pers.columns.isin(['target'])] #X_test[y_test[y_test == 'Carroll'].index]
y_test_Pers = subsample_Pers['target']

In [173]:
# concatenate test sets to full Carroll-nonCarroll test set
X_test_AnonA = np.concatenate([X_test_Carroll, X_test_emma, X_test_Pers], axis=0)
y_test_AnonA = np.concatenate([y_test_Carroll, y_test_emma, y_test_Pers], axis=0)

In [174]:
# run existing model over new test set
print('\nTest set score:', lr.score(X_test_AnonA, y_test_AnonA))
lr_AnA_predicted = lr.predict(X_test_AnonA)
pd.crosstab(y_test_AnonA, lr_AnA_predicted)


Test set score: 0.85


col_0    Austen  Carroll
row_0                   
Austen      125       31
Carroll      14      130